In [1]:
#to run if necessary
#!pip install catboost
#!pip install xgboost
#!pip install lightgbm

# 0.Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from  xgboost import XGBRegressor 
import os
import time
import pickle
import math
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor



# 1.Import Dataset

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


# Charger votre DataFrame
df = pd.read_csv('datasets/yann_train_avec_index.csv', sep=';')

# Séparer les features et la target
X = df.drop(['index','Unnamed: 0', 'Month 4','Third_T1','Third_T2','Third_T3'], axis=1)
y = df['Month 4']

# Effectuer le train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création de l'imputateur
imputer = SimpleImputer(strategy='mean')

# Imputation sur les ensembles d'entraînement et de test
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Recombiner les features et les targets pour le train et le test
train = pd.concat([X_train_imputed, y_train.reset_index(drop=True)], axis=1)
test = pd.concat([X_test_imputed, y_test.reset_index(drop=True)], axis=1)

# Sauvegarder les ensembles de données train et test en format CSV
train.to_csv('datasets/yann/train_data.csv', index=False)
test.to_csv('datasets/yann/test_data.csv', index=False)


###     1.1. Standardize train and test before training

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Initialize the scaler
scaler = StandardScaler()

# Assume 'train' and 'test' are your initial DataFrames and 'Month 4' is a column not an index

# Separate features and target variable for training data
X_train = train.drop('Month 4', axis=1)
y_train = train['Month 4'].copy()

# Separate features and target variable for test data
X_test = test.drop('Month 4', axis=1)
y_test = test['Month 4'].copy()

# Fit the scaler on the training features
scaler.fit(X_train)

# Scale the training features
X_train_scaled = scaler.transform(X_train)

# Scale the test features
X_test_scaled = scaler.transform(X_test)

# Convert the scaled arrays back into DataFrames
X_train = pd.DataFrame(X_train_scaled, index=train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, index=test.index, columns=X_test.columns)

# Concatenate the scaled features with the target variable
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)


# 2. CREATION ON THE MODELS AND MODEL SELECTION

# 2.1. creation of function 

In [37]:
import time
import pickle
import math
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Importez ici les modèles spécifiques que vous utilisez


from sklearn.metrics import mean_squared_error

def rmse_score(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def RMSE_hfactory(y_true, y_pred):
    # Calculez R_0 en utilisant la valeur vraie et une liste de zéros
    R_0 = rmse_score(y_true, [0]*len(y_true))
    
    # Calculez RMSE_pred en utilisant la valeur vraie et les prédictions du modèle
    RMSE_pred = rmse_score(y_true, y_pred)
    
    # Calculez RMSE_hfactory en utilisant la formule donnée
    RMSE_hfactory_value = (R_0 - 0.8 * RMSE_pred) / R_0
    
    return RMSE_hfactory_value


def train_prediction_models(data_path_suffixe, train, test, model_names, models, hyperparameters):
    not_keep = ['Unnamed: 0','Month 4']
    
    print("--------------------------------- \n" +
          "TRAINING IN PROCESS  : " + data_path_suffixe
          )

    # Récupération des hyper-paramètres optimaux
    best_params = {}

    # Variables utilisées dans le modèle
    predictors = [p for p in train.columns if p not in not_keep]
    target = 'Month 4'

    # Split train / test
    X_train = train[predictors]
    y_train = train[target]

    X_test = test[predictors]
    y_test = test[target]

    # Entraînement et prédiction des modèles
    for model_name in model_names:
        print('----------------------------------------------------')
        print(model_name)

        start = time.time()

        model = models[model_name]
        hyperparameters_model = hyperparameters[model_name]
        search = GridSearchCV(model, hyperparameters_model, cv=5, scoring=make_scorer(rmse_score, greater_is_better=False), verbose=0, n_jobs=-1)
        search.fit(X_train, y_train)

        best_estimator = search.best_estimator_
        print('Hyperparameters:', search.best_params_)

        # Sauvegarde des hyperparamètres optimaux
        for param in search.best_params_:
            best_params[model_name + '__' + param] = search.best_params_[param]

        # Prédiction et évaluation
        y_pred_train = best_estimator.predict(X_train)
        y_pred_test = best_estimator.predict(X_test)
        mse_train = rmse_score(y_train, y_pred_train)
        mse_test = rmse_score(y_test, y_pred_test)
        
        rmseh_train = RMSE_hfactory(y_train, y_pred_train)
        rmseh_test = RMSE_hfactory(y_test, y_pred_test)

        print('train RMSE Hfactory:', rmseh_train)
        print('test RMSE Hfactory :', rmseh_test)

        print('Train RMSE:', mse_train)
        print('Test RMSE:', mse_test)

        end = time.time()
        print('Execution time:', math.ceil((end - start) / 60), 'min')
        print('----------------------------------------------------')
        print(' ')

        # Génération des noms de fichiers avec un compteur en cas de fichiers existants
        counter = 0
        while True:
            model_filename = f'model_{model_name}_{counter}.pkl'
            params_filename = f'hyperparameters_{model_name}_{counter}.txt'

            # Vérifier si les fichiers existent déjà
            if not os.path.exists(model_filename) and not os.path.exists(params_filename):
                # Sauvegarde du modèle
                with open(model_filename, 'wb') as file:
                    pickle.dump(best_estimator, file)
                
                # Sauvegarde des hyperparamètres
                with open(params_filename, 'w') as file:
                    file.write('Best Hyperparameters for ' + model_name + ':\n')
                    file.write(str(search.best_params_))
                break
            else:
                counter += 1
    # Résumé des hyper-paramètres optimaux des modèles
    print('Optimal hyperparameters of models')
    print(best_params)


# 2.2 Test of different models

# Tous les modèles

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

seed = 42
np.random.seed(seed)

# Model names
model_names = ['LinearRegression', 'CatBoost', 'XGBoost', 'RandomForest', 'DecisionTree', 'VotingRegressor']

# Models used
models = {
    'CatBoost': CatBoostRegressor(verbose=False, random_state=seed),
    'XGBoost': XGBRegressor(verbose=False, random_state=seed),
    'RandomForest': RandomForestRegressor(verbose=False, random_state=seed),
    'DecisionTree': DecisionTreeRegressor(random_state=seed),
    'LinearRegression': LinearRegression(),
    'VotingRegressor': VotingRegressor(estimators=[
        ('CatBoost', CatBoostRegressor(verbose=False, random_state=seed)), 
        ('XGBoost', XGBRegressor(verbose=False, random_state=seed)), 
        ('RandomForest', RandomForestRegressor(verbose=False, random_state=seed))
    ])
}

# Hyperparameter grids
hyperparameters = {
    'CatBoost': {'n_estimators': list(range(1000, 2550, 150)), 'learning_rate': [0.001, 0.01, 0.05, 0.1]},
    'XGBoost': {'n_estimators': list(range(1000, 1500, 150)), 'learning_rate': [0.001]},
    'RandomForest': {'n_estimators': list(range(1000, 2550, 150)), 'max_depth': list(range(2, 11)) + [None]},
    'DecisionTree': {'max_depth': list(range(2, 11)) + [None]},
    # For LinearRegression, you may not need to define any hyperparameters, or you can choose other relevant ones
    'LinearRegression': {},
    'VotingRegressor': {
        'CatBoost__n_estimators': [1000], 
        'CatBoost__learning_rate': [0.05], 
        'XGBoost__n_estimators': [1000], 
        'XGBoost__learning_rate': [0.05], 
        'RandomForest__n_estimators': [2000], 
        'RandomForest__max_depth': [None]
    }
}

# Call the training function
train_prediction_models("dataset/data", train, test, model_names, models, hyperparameters)


--------------------------------- 
TRAINING IN PROCESS FOR DISEASE : dataset/data
----------------------------------------------------
LinearRegression
Hyperparameters: {}
Train MSE: 297957.3150024991
Test MSE: 376335.60683266533
Execution time: 1 min
----------------------------------------------------
 
----------------------------------------------------
CatBoost
Hyperparameters: {'learning_rate': 0.001, 'n_estimators': 2500}
Train MSE: 242961.55378075698
Test MSE: 438594.50243551907
Execution time: 266 min
----------------------------------------------------
 
----------------------------------------------------
XGBoost


# MLP

In [ ]:
from sklearn.neural_network import MLPRegressor

# Définir les noms des modèles et les modèles eux-mêmes
model_names = ['MLPRegressor']
models = {
    'MLPRegressor': MLPRegressor()
}

# Définir les hyperparamètres pour la recherche sur grille
hyperparameters = {
    'MLPRegressor': {
        'hidden_layer_sizes': [(100,), (100,100)],
        'alpha': [0.001],
        'max_iter': [200],
        'solver': ['adam']  # Utilisation explicite de l'optimiseur Adam
    }
}

# Appel de la fonction
train_prediction_models("dataset/data", train, test, model_names, models, hyperparameters)


# Elastic Search

In [ ]:
from sklearn.linear_model import ElasticNet

# Définir les noms des modèles et les modèles eux-mêmes
model_names = ['ElasticNet']
models = {
    'ElasticNet': ElasticNet()
}

# Définir les hyperparamètres pour la recherche sur grille
hyperparameters = {
    'ElasticNet': {'alpha': [1], 'l1_ratio': [0.8]}
}

# Appel de la fonction
train_prediction_models("dataset/data" , train, test, model_names, models, hyperparameters)


--------------------------------- 
TRAINING IN PROCESS FOR DISEASE : dataset/data
----------------------------------------------------
ElasticNet
Hyperparameters: {'alpha': 1, 'l1_ratio': 0.8}
Train RMSE: 0.37667140845113095
Test RMSE: 0.40028055734684986
Train RMSE: 547.5269436325527
Test RMSE: 619.1318458271661
Execution time: 1 min
----------------------------------------------------
 
Optimal hyperparameters of models
{'ElasticNet__alpha': 1, 'ElasticNet__l1_ratio': 0.8}


# Xgboost

In [ ]:
# Définir les noms des modèles et les modèles eux-mêmes
model_names = ['XGBoost']
models = {'XGBoost': XGBRegressor()}

# Définir les hyperparamètres pour la recherche sur grille
hyperparameters = {'XGBoost': {'alpha': [0.6378671093136276],
 'colsample_bytree': [0.717229517496302],
 'gamma': [0.22950133682316104],
 'learning_rate': [0.014810947307879158],
 'max_depth': [5],
 'min_child_weight': [2],
 'subsample': [0.7507159458738991]}
}



# Appel de la fonction
train_prediction_models("dataset/data" , train, test, model_names, models, hyperparameters)

# 3. Feature selection from feature importance

In [18]:
# Load the pickled XGBoost model
with open('model_XGBoost_1.pkl', 'rb') as file:
    xgb_model = pickle.load(file)

# Get feature importance
feature_importance = xgb_model.get_booster().get_score(importance_type='weight')

# Display feature importance
print("Feature Importance:")
for feature, importance in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True):
    print(f"{feature}: {importance}")

# define a threshold
seuil_importance = 4
features_to_keep = [feature for feature, importance in feature_importance.items() if importance >= seuil_importance]

# Filtrer les DataFrames d'entraînement et de test
X_train_filtered = X_train[features_to_keep]
X_test_filtered = X_test[features_to_keep]

features_to_keep
features_to_keep.append('Month 4')


Feature Importance:
Month 3: 216.0
Month 2: 174.0
Month 1: 170.0
Final consumption expenditure (annual % growth): 22.0
Exports of goods and services (annual % growth): 11.0
Energy Price Index: 11.0
Services, value added (annual % growth): 9.0
population (2023): 8.0
Industry (including construction), value added (annual % growth): 8.0
Headline Consumer Price Index: 8.0
Agriculture, forestry, and fishing, value added (annual % growth): 5.0
GDP (current US$): 5.0
Imports of goods and services (annual % growth): 3.0
Infrastructure Score: 2.0
Manufacturing, value added (annual % growth): 2.0
Division proxy_Division-2: 2.0
netChange: 1.0
Timeliness Score: 1.0
Tracking and Tracing Score: 1.0
Division proxy_Division-1: 1.0
Product Life cycel status_EOL: 1.0
Product Life cycel status_active: 1.0


# 4. Final modele using features selection

In [42]:
# Définir les noms des modèles et les modèles eux-mêmes
model_names = ['XGBoost']
models = {'XGBoost': XGBRegressor()}

# Définir les hyperparamètres pour la recherche sur grille
hyperparameters = {'XGBoost': {'alpha': [0.70],
 'colsample_bytree': [0.717229517496302],
 'gamma': [0.22950133682316104],
 'learning_rate': [0.014810947307879158],
 'max_depth': [5],
 'min_child_weight': [2],
 'subsample': [0.7507159458738991]}
}


# Appel de la fonction
train_prediction_models("dataset/data" ,train[features_to_keep], test[features_to_keep], model_names, models, hyperparameters)

--------------------------------- 
TRAINING IN PROCESS FOR DISEASE : dataset/data
----------------------------------------------------
XGBoost


is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


Hyperparameters: {'alpha': 0.7, 'colsample_bytree': 0.717229517496302, 'gamma': 0.22950133682316104, 'learning_rate': 0.014810947307879158, 'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.7507159458738991}


is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


train RMSE Hfactory: 0.4955386618615962
test RMSE Hfactory : 0.4320364163556861
Train RMSE: 443.1148809737429
Test RMSE: 586.3480769418557
Execution time: 4 min
----------------------------------------------------
 
Optimal hyperparameters of models
{'XGBoost__alpha': 0.7, 'XGBoost__colsample_bytree': 0.717229517496302, 'XGBoost__gamma': 0.22950133682316104, 'XGBoost__learning_rate': 0.014810947307879158, 'XGBoost__max_depth': 6, 'XGBoost__min_child_weight': 2, 'XGBoost__subsample': 0.7507159458738991}


# 5. Creation of file for submission

In [ ]:
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


# import joblib  # Utilisez cette importation si vous préférez utiliser joblib pour charger le modèle

# Charger les données de test normalisées
X_test_avec_index = pd.read_csv('datasets/yann_test_avec_index.csv', sep=';')
X_test_avec_index = X_test_avec_index.drop(['Unnamed: 0','Third_T1','Third_T2','Third_T3'], axis=1)
index = X_test_avec_index["index"].copy()
X_test_sans_index = X_test_avec_index.drop(columns = 'index')

# Créer un imputer qui remplacera les valeurs manquantes par la moyenne des colonnes
imputer = SimpleImputer(strategy='mean')
# Adapter l'imputer aux données et transformer les données
X_test_sans_index_mean = imputer.fit_transform(X_test_sans_index)
# Convertir le résultat en DataFrame et remettre 'Month 4' comme index
X_test = pd.DataFrame(X_test_sans_index_mean, index=X_test_sans_index.index, columns=X_test_sans_index.columns)


# scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_train = pd.read_csv('datasets/yann/train_data.csv', sep=',').drop('Month 4', axis = 1)

scaler.fit(train_train)

# Transformer les données de test avec le même scaler
X_test_scaled = scaler.transform(X_test)

# Convertir le résultat en DataFrame et remettre l'index
X_test_scaled = pd.DataFrame(X_test_scaled, index=index, columns=X_test.columns)


# prediction

In [ ]:

features_to_keep = ['Month 1',
 'Month 2',
 'Month 3',
 'population (2023)',
 'Agriculture, forestry, and fishing, value added (annual % growth)',
 'Exports of goods and services (annual % growth)',
 'Final consumption expenditure (annual % growth)',
 'GDP (current US$)',
 'Industry (including construction), value added (annual % growth)',
 'Services, value added (annual % growth)',
 'Energy Price Index',
 'Headline Consumer Price Index']

# Charger le modèle depuis le fichier pickle
with open('datasets/yann/model_XGBoost_good.pkl', 'rb') as file:  # Remplacer par le chemin de votre modèle
    model = pickle.load(file)
    # ou avec joblib : model = joblib.load('path_to_your_model.pkl')

    
# Utiliser le modèle pour faire des prédictions
predictions = model.predict(X_test_scaled[features_to_keep])

# Afficher les prédictions
print(predictions)


predictions_df = pd.DataFrame(predictions, columns=['Month 4'])
predictions_df['index'] = index
predictions_df


# Sauvegarder les prédictions dans un fichier CSV, 'Month 4' comme index
predictions_df.to_csv('datasets/predictions.csv', sep=';')


In [ ]:
plt.plot(predictions_df['Month 4'])

# save file

In [ ]:
df = pd.read_csv("datasets/predictions.csv", delimiter=";")
df_test = df.drop(columns=["Unnamed: 0"])
df_test = df_test[["index", "Month 4"]]
df_test.to_csv("predictions_example_6666.csv", sep=";" , index=False)